In [20]:
import streamlit as st
import pandas as pd
import folium as fol
from streamlit_folium import st_folium
from folium.plugins import MarkerCluster

In [24]:
# CSV einlesen mit robustem Parser
df = pd.read_csv("C:/Users/PC/Desktop/Abschlussprojekt/data/electric_vehicles/Ladesaeulenregister_BNetzA.CSV", sep=';', encoding='ISO-8859-1', dtype=str)

In [25]:
relevante_spalten = ["Breitengrad", "Längengrad", "Ort", "Straße", "Hausnummer", "Art der Ladeeinrichtung"]

In [26]:
# Fehlende Werte entfernen
df = df[relevante_spalten].dropna(subset=["Breitengrad", "Längengrad"])

In [27]:
# Funktion: Nur Werte behalten, die nur aus Ziffern und maximal einem Punkt bestehen
def koordinaten_bereinigen(df):
    def ist_gueltig(val):
        val = str(val).replace('.', '')
        return val.isdigit()
    
    def format_breitengrad(val):
        val = str(val).replace('.', '')
        val = val.ljust(8, '0')[:8]
        return f"{val[:-6]}.{val[-6:]}"  # z.B. 51805270 → 51.805270

    def format_laengengrad(val):
        val = str(val).replace('.', '')
        if val.startswith("1"):
            val = val.ljust(8, '0')[:8]
        else:
            val = val.ljust(7, '0')[:7]
        return f"{val[:-6]}.{val[-6:]}"  # z.B. 10332870 → 10.332870
    
    df = df[df["Breitengrad"].apply(ist_gueltig)]
    df = df[df["Längengrad"].apply(ist_gueltig)]

    df["Breitengrad"] = df["Breitengrad"].apply(format_breitengrad).astype(float)
    df["Längengrad"] = df["Längengrad"].apply(format_laengengrad).astype(float)
    
    return df

df = koordinaten_bereinigen(df)

In [28]:
# Nur Koordinaten in Deutschland behalten
df = df[
    df["Breitengrad"].between(47.0, 55.1) &
    df["Längengrad"].between(5.5, 15.5)
]

print(f"Aktuell verbleibende Datensätze: {len(df)}")

Aktuell verbleibende Datensätze: 88575


In [30]:
# Karte erzeugen
m = fol.Map(location=[51.1657, 10.4515], zoom_start=6)
marker_cluster = MarkerCluster().add_to(m)

In [33]:
# Marker hinzufügen (hier: max. 10.000 für Performance)
for _, row in df.head(89000).iterrows():
    popup_text = f"{row.get('Ort', '')}, {row.get('Straße', '')} {row.get('Hausnummer', '')}"
    fol.Marker(
        location=[row["Breitengrad"], row["Längengrad"]],
        popup=popup_text
    ).add_to(marker_cluster)

In [34]:
# Karte speichern
st_folium(m, width=800, height=600)

2025-06-03 22:19:30.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 22:19:30.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 22:19:30.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 22:19:30.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'last_clicked': None,
 'last_object_clicked': None,
 'last_object_clicked_tooltip': None,
 'last_object_clicked_popup': None,
 'all_drawings': None,
 'last_active_drawing': None,
 'bounds': {'_southWest': {'lat': 47.352641, 'lng': 5.895952},
  '_northEast': {'lat': 55.0196, 'lng': 14.999334}},
 'zoom': 6,
 'last_circle_radius': None,
 'last_circle_polygon': None,
 'selected_layers': None}

In [ ]:
# zum ausführen muss die folgende Bibliothek installiert werden
# pip install streamlit folium streamlit-folium pandas